In [11]:
'''
Visualization of the filters of a CNN, via gradient ascent in input space.
This script can run on CPU in a few minutes.
This script is meant for use only and is not discussed in great details as it is beyond the scope of the class.
'''
# using tf.2.1 in colab
%tensorflow_version 2.x

from __future__ import print_function

import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras import backend as K
from google.colab import drive
# stopping eager execution mode to be allowed to use the gradient function
tf.compat.v1.disable_eager_execution()
# sanity check for tf version
print(tf.__version__)

###################################################################################################
# dimensions of the generated pictures for each filter.
###################################################################################################   
img_width = 150
img_height = 150

###################################################################################################
# the name of the layer we want to visualize 
###################################################################################################   
layer_name='conv2d_4'

###################################################################################################
# util function to convert a tensor into a valid image
###################################################################################################   
def deprocess_image(x):
    
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

###################################################################################################
# Load the model
###################################################################################################   
from tensorflow.keras.models import load_model
drive.mount('/content/gdrive')
model=load_model('/content/gdrive/My Drive/cse-30321-lab2/dogs_cats_birds_model_case02_30epoch')

###################################################################################################
# This is the placeholder for the input images
###################################################################################################   
input_img = model.input

###################################################################################################
# Get the symbolic outputs of each "key" layer (we gave them unique names).
###################################################################################################   
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

###################################################################################################
# Utility function to normalize a tensor by its L2 norm
###################################################################################################   
def normalize(x): 
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

kept_filters = []

###################################################################################################
# Scan through some number of filters...
###################################################################################################   

for filter_index in range(64):

    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(100):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)

    # decode the resulting input image
    if True:
        
    #if loss_value > 0
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

###################################################################################################
# we will stich the best n^2 filters on a n x n grid.
###################################################################################################
n = 5

###################################################################################################
# the filters that have the highest loss are assumed to be more intuitive
# we will only keep the top n filters.
###################################################################################################   
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

###################################################################################################
# build a black picture with enough space for
# our n x n filters of size 128 x 128, with a 5px margin in between
###################################################################################################   
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

###################################################################################################
# fill the picture with our saved filters 
###################################################################################################   
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

###################################################################################################
# save the result to drive
###################################################################################################   
save_img('/content/gdrive/My Drive/cse-30321-lab2/dogs_cats_birds_model_case02_30epoch_%dx%d.png' % (n, n), stitched_filters)
print('saved')

Streaming output truncated to the last 5000 lines.
Filter 14 processed in 6s
Processing filter 15
Current loss value: 52.91431
Current loss value: 67.71484
Current loss value: 82.366684
Current loss value: 99.86915
Current loss value: 125.47368
Current loss value: 154.84966
Current loss value: 185.1908
Current loss value: 217.06314
Current loss value: 252.34094
Current loss value: 291.67206
Current loss value: 337.86453
Current loss value: 385.40356
Current loss value: 440.1584
Current loss value: 508.8334
Current loss value: 581.247
Current loss value: 650.3296
Current loss value: 730.08026
Current loss value: 816.14343
Current loss value: 910.1655
Current loss value: 1003.1953
Current loss value: 1099.4175
Current loss value: 1197.6477
Current loss value: 1296.3727
Current loss value: 1397.4824
Current loss value: 1514.1699
Current loss value: 1627.6077
Current loss value: 1740.5393
Current loss value: 1848.4873
Current loss value: 1965.1455
Current loss value: 2078.2712
Current loss